In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, AdamW
from tensorflow.keras import regularizers, models, layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import optuna
import wandb
import gc
import cv2

import os
import sqlite3

In [ ]:
tf.keras.backend.clear_session()

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("TensorFlow is using the GPU \n", gpus)
else:
    print("No GPU detected.")

In [ ]:
import wandb 

from wandb.integration.keras import WandbMetricsLogger

wandb.require("core")
wandb.login()

In [ ]:
# We create the dataframe for all the data subsets for training, test and validation
sg = pd.read_csv("/tf/Higgs-Boson-LHC-Collision-Detector/Datasets/Signal.csv")
bg = pd.read_csv("/tf/Higgs-Boson-LHC-Collision-Detector/Datasets/Background.csv")

sg_sample1 = sg.sample(frac = 0.5, random_state = 4)
sg_sample2 = sg.sample(frac = 0.3, random_state = 4)
sg_sample3 = sg.sample(frac = 0.2, random_state = 4)

bg_sample1 = bg.sample(frac = 0.5, random_state = 4)
bg_sample2 = bg.sample(frac = 0.3, random_state = 4)
bg_sample3 = bg.sample(frac = 0.2, random_state = 4)

train = pd.concat([sg_sample1, bg_sample1])
test = pd.concat([sg_sample2, bg_sample2])
val = pd.concat([sg_sample3, bg_sample3])

# Now we can create the subsets:
X_train, X_test, X_val = train.drop(columns = ["label"]), test.drop(columns = ["label"]), val.drop(columns = ["label"])
y_train, y_test, y_val = train["label"], test["label"], val["label"]

# And we stardardize the data:

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Check the sizes
print("Train size:", len(X_train))
print("Validation size:", len(X_val))
print("Test size:", len(X_test))

In [ ]:
# For the trials we make a smaller train and test sets:

train_small = train.sample(frac = 0.15, random_state = 5)
test_small = test.sample(frac = 0.15, random_state = 5)

X_train_small, X_test_small = train_small.drop(columns = ["label"]), test_small.drop(columns = ["label"])
y_train_small, y_test_small = train_small["label"], test_small["label"]

X_train_small = scaler.fit_transform(X_train_small)
X_test_small = scaler.transform(X_test_small)

# Check the sizes
print("Train size:", len(X_train_small))
print("Test size:", len(X_test_small))

In [ ]:
X_train_small.shape

In [ ]:
# We define a function to create residual blocks (considering a layer regularizer L2)

def residual_block(x, n, activation, dropout, dropout_rate, regularizer, r_2):
        
    residual = x  
        
    if dropout == "y":
        # Camino "principal"
        x = layers.Dense(n, activation = activation, kernel_regularizer = regularizers.l2(r_2))(x)
        x = layers.BatchNormalization()(x)
        
        # Capa intermedia Dropot
        x = layers.Dropout(dropout_rate)(x)
            
        # Capa lineal
        x = layers.Dense(n, kernel_regularizer = regularizers.l2(r_2))(x)
        x = layers.BatchNormalization()(x)
            
    else: 
        # Camino "principal"
        x = layers.Dense(n, activation = activation, kernel_regularizer = regularizers.l2(r_2))(x)
        x = layers.BatchNormalization()(x)
            
        # Capa lineal
        x = layers.Dense(n, kernel_regularizer = regularizers.l2(r_2))(x)
        x = layers.BatchNormalization()(x)

    # Suma de la conexión residual
    x = layers.add([x, residual]) 
    x = layers.Activation(activation)(x)
        
    return x

In [ ]:
# We create the function to start the study and optimizarion using Optuna
def objective(trial):

    tf.keras.backend.clear_session()

    inputs = layers.Input(shape = (X_train.shape[1],))
    
    #############################################################################################################
    
    # Optuna suggests activation function for all layers
    activation = trial.suggest_categorical(f"activation_L{i+2}", ["relu", "relu6", "leaky_relu"])
    
    # Optuna suggests regularizer L2 value
    regularizer = "L2"
    r_2 = trial.suggest_float("regularizer_value_2", 1e-7, 1e-5, log = True)
    
    # Optuna suggest the number of layers
    n_layers = trial.suggest_int("N_layers", 15,20)
    
    # Optuna suggests learning rate value and an optimizer
    lr = trial.suggest_float("learning_rate", 2.5e-4, 1e-3, log = True)
    
    optimizer_name = trial.suggest_categorical("optimizer", ["sgd", "adam", "rmsprop", "adamw"])
                              
    if optimizer_name == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate = lr)
    elif optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
    elif optimizer_name == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr)
    elif optimizer_name == "adamw":
        optimizer = tf.keras.optimizers.AdamW(learning_rate = lr)              
    
    #############################################################################################################
    
    # First layer

    # Optuna suggest number of neurons for the first layer

    N = trial.suggest_int("N_1st_layer", 128, 256)
    
    x = layers.Dense(N, input_shape = (X_train.shape[1],))(inputs)
    x = layers.Activation(activation)(x)
    x = layers.BatchNormalization()(x)
    
    # Optuna suggests neurons for the residual blocks and if using Dropout block
    
    dropout_per_layer = []
    dropout_percentage_per_layer = []
            
    dropping_out = trial.suggest_categorical("Dropout", ["y", "n"])

    N_per_layer = []
    
    for i in range(n_layers):
        
        n = trial.suggest_int(f"N_{i+1}_layer", 128, 256)
        N_per_layer.append(n)
                              
        dropout_rate = trial.suggest_float(f"Dropout_value_L{i+2}",0.1, 0.15)
        
        # i-th residual block:
        
        # Choosing between Dropout or a regulizer
        
        if dropping_out == "y":
            dropout_percentage_per_layer.append(dropout_rate)
            x = residual_block(x, n, activation, dropout, dropout_rate, regularizer, r_2)

        else:
            dropout_percentage_per_layer.append(0.0)
            x = residual_block(x, n, activation, dropout, dropout_rate, regularizer, r_2)            
            
    x = layers.Dropout(0.4)(x)  
    outputs = layers.Dense(1, activation = "sigmoid")(x)
    model = models.Model(inputs, outputs)
                              
    model.compile(optimizer = optimizer,
                  loss = "binary_crossentropy",
                  metrics = ["accuracy",
                             tf.keras.metrics.Precision(),
                             tf.keras.metrics.AUC(curve = "ROC"),
                             tf.keras.metrics.AUC(curve = "PR")])
    
    #############################################################################################################

    wandb.init(
        project = "Residual-SnB-Trials-1.0",
        name = f"Trial_{trial.number}",
        reinit = True,
        config = {
            "Units_1": N,
            "Units_per_layer": N_per_layer,
            "activation": activation,
            "n_layers": n_layers,
            "regularizer": regularizer,
            "r_value2": r_2,
            "Dropout": dropping_out, 
            "dropout_percentage_per_layer": dropout_percentage_per_layer,
            "learning_rate": lr,
            "optimizer": activation,
        }
    )
    
    #############################################################################################################
    
    """
    Callbacks
    """
    early_stopping = EarlyStopping(monitor = "val_accuracy", patience = 10, restore_best_weights = True)
    lr_reduction = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 7)
#     tensorboard_cb = TensorBoard(log_dir = "/workspace/Optuna-Trials/Plant-Pathology-Classificator/tf_debug", histogram_freq = 1, write_graph = True,
#                                  write_images = False)
    
    #############################################################################################################
    
    """
    Creación del modelo
    """
    
    try:
        print(model.summary())
    
        history = model.fit(
            X_train_small, y_train_small,
            validation_data = (X_test_small, y_test_small),
            epochs = 200,
            verbose = 1, 
            callbacks = [WandbMetricsLogger(log_freq = 5), early_stopping, lr_reduction]
        )

        val_loss = min(history.history["val_loss"])
        val_accuracy = max(history.history["val_accuracy"])
        
        train_loss = min(history.history["loss"])
        train_accuracy = max(history.history["accuracy"])
    
    except tf.errors.ResourceExhaustedError as e:
        
        print(f"Intento {trial.number} falló debido a: {e}")
        
        tf.keras.backend.clear_session()
        wandb.finish()
        gc.collect()
        
        return float("inf")

    except Exception as e:
        
        print(f"Intento {trial.number} falló. Unexpected error: {e}")
        
        tf.keras.backend.clear_session()
        wandb.finish()
        gc.collect()
        
        return float("inf")
    
    # score = val_loss + 0.1 * (train_loss - val_loss)
    
    score = val_accuracy
    
    # score = train_loss 
    
    tf.keras.backend.clear_session()
    gc.collect()
    wandb.finish()

    return 1-score

In [ ]:
study = optuna.create_study(
    study_name = "Residual-Trials-1.0",
    direction = "minimize",
    storage = "sqlite:////workspace/Optuna-Trials/ResNet_SnB_study.db",
    load_if_exists = True
)

study.optimize(objective, n_trials = 500)